In [8]:
pwd

'/home/wsuser/work'

In [9]:
!pip install keras
!pip install tensorflow

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [42]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen =ImageDataGenerator(rescale = 1)

In [43]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_20ac6834bfba4a38a284a4244a15275d = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_20ac6834bfba4a38a284a4244a15275d = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_20ac6834bfba4a38a284a4244a15275d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Yyg4bMEcaIRgTDvY1vLe4MRe0hFws2TxXmg6oBdIjjtu',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_20ac6834bfba4a38a284a4244a15275d)

streaming_body_1 = client_20ac6834bfba4a38a284a4244a15275d.get_object(Bucket='plantdiseasedetectionnew-donotdelete-pr-v5qf5xs6hkyvwm', Key='Veg-dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [44]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [45]:
pwd

'/home/wsuser/work'

In [46]:
import os
filenames = os.listdir('/home/wsuser/work/Veg-dataset')

In [47]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Veg-dataset/train_set', 
                                            target_size = (128,128), 
                                            batch_size = 16, 
                                            class_mode = 'categorical')

Found 11386 images belonging to 9 classes.


In [48]:
x_test = test_datagen.flow_from_directory('/home/wsuser/work/Veg-dataset/test_set', 
                                          target_size = (128,128), 
                                          batch_size = 16, 
                                          class_mode = 'categorical')

Found 3416 images belonging to 9 classes.


In [49]:
print(x_train.class_indices)

{'Pepper,_bell___Bacterial_spot': 0, 'Pepper,_bell___healthy': 1, 'Potato___Early_blight': 2, 'Potato___Late_blight': 3, 'Potato___healthy': 4, 'Tomato___Bacterial_spot': 5, 'Tomato___Late_blight': 6, 'Tomato___Leaf_Mold': 7, 'Tomato___Septoria_leaf_spot': 8}


In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [50]:
model = Sequential()

In [51]:
model.add(Convolution2D(32,(3,3), input_shape = (128,128,3), activation = 'relu'))

In [52]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [53]:
model.add(Flatten())

In [55]:
model.add(Dense(units = 300, kernel_initializer ='uniform', activation ='relu'))

In [56]:
model.add(Dense(units = 150, kernel_initializer ='uniform', activation ='relu'))

In [57]:
model.add(Dense(units = 75, kernel_initializer ='uniform', activation ='relu'))

In [61]:
model.add(Dense(units = 9,activation = 'softmax',kernel_initializer ='uniform'))

In [62]:
model.compile(loss = 'categorical_crossentropy',optimizer = "adam",metrics = ["accuracy"])

In [63]:
model.fit_generator(x_train, steps_per_epoch = 89,
                    epochs = 20,
                    validation_data = x_test,
                    validation_steps = 27)

Epoch 1/20
89/89 [==============================] - 48s 537ms/step - loss: 2.1171 - accuracy: 0.1700 - val_loss: 146.5959 - val_accuracy: 0.3125
Epoch 2/20
89/89 [==============================] - 49s 546ms/step - loss: 1.7052 - accuracy: 0.3384 - val_loss: 174.9114 - val_accuracy: 0.3611
Epoch 3/20
89/89 [==============================] - 49s 555ms/step - loss: 1.4568 - accuracy: 0.4809 - val_loss: 354.2682 - val_accuracy: 0.2361
Epoch 4/20
89/89 [==============================] - 47s 527ms/step - loss: 1.2222 - accuracy: 0.5587 - val_loss: 499.4476 - val_accuracy: 0.3125
Epoch 5/20
89/89 [==============================] - 48s 537ms/step - loss: 1.2538 - accuracy: 0.5555 - val_loss: 460.1074 - val_accuracy: 0.3148
Epoch 6/20
89/89 [==============================] - 48s 536ms/step - loss: 1.0635 - accuracy: 0.6024 - val_loss: 436.2907 - val_accuracy: 0.3542
Epoch 7/20
89/89 [==============================] - 47s 528ms/step - loss: 1.0280 - accuracy: 0.6150 - val_loss: 331.5797 - val_ac

In [64]:
model.save('vegetable.h5')

In [65]:
!tar -zcvf image-classification-model_new.tgz vegetable.h5

vegetable.h5


In [66]:
ls -1

image-classification-model_new.tgz
Veg-dataset/
vegetable.h5


In [67]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 23.8 MB/s eta 0:00:01


In [68]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                  "url": "https://us-south.ml.cloud.ibm.com",
                  "apikey": "uTSfzeCSWD4gQprdLNy_Q4JGQH-jrt3-EErGuSe8LOWu"
                  }
client = APIClient(wml_credentials)

In [69]:
client = APIClient(wml_credentials)

In [70]:
def guid_from_space_name(client, space_name):
        space = client.spaces.get_details()
        return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [71]:
space_uid = guid_from_space_name(client, 'vegetabletraining')
print("Space UID = " + space_uid)

Space UID = f396560a-414c-4544-a578-cddb4e8f2e18


In [72]:
client.set.default_space(space_uid)

'SUCCESS'

In [73]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [74]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid

'2b73a275-7cbf-420b-a912-eae7f436e0bc'

In [75]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [76]:
model_id

'a350f994-6c3e-44cc-920b-0f66ca31d45b'

In [77]:
client.repository.download(model_id,'my_model1.tar.gz')

Successfully saved model content to file: 'my_model1.tar.gz'


'/home/wsuser/work/my_model1.tar.gz'

In [79]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2

In [82]:
model = load_model("vegetable.h5")

In [84]:

streaming_body_2 = client_20ac6834bfba4a38a284a4244a15275d.get_object(Bucket='plantdiseasedetectionnew-donotdelete-pr-v5qf5xs6hkyvwm', Key='b8e606e2-0630-453f-8688-b4350feb2bce___GCREC_Bact.Sp 3033.JPG')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
img = image.load_img(streaming_body_2,target_size = (64,64))

TypeError: expected str, bytes or os.PathLike object, not StreamingBody

In [50]:
img = image.load_img(r'./leaf disease/test_set/Tomato___Bacterial_spot/b8e606e2-0630-453f-8688-b4350feb2bce___GCREC_Bact.Sp 3033.JPG',
                     target_size = (128,128))
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
x.shape

(1, 128, 128, 3)

In [51]:
pred = model.predict_classes(x)
pred

array([8], dtype=int64)